In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [4]:
# Import libraries
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

Téléchargement des data :

In [9]:
train_dataframes = []
for i in tqdm(range(1, 3)):
    train_dataframes.append(pd.read_csv(f'data-train/train_data_part_{i}.csv'))
train_data = pd.concat(train_dataframes, ignore_index=True)

# free up memory by deleting the dataframes we no longer need
del train_dataframes

100%|██████████| 2/2 [00:52<00:00, 26.22s/it]


In [6]:
# This code reads the data from a CSV file named "products_data.csv" into a pandas DataFrame
products_data = pd.read_csv('data-train/products_data.csv')

C:\Users\jeann\AppData\Local\Temp\ipykernel_15484\1971356203.py:2: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  products_data = pd.read_csv('data-train/products_data.csv')


In [10]:
# This code reads the data from a CSV file named "test_data.csv" into a pandas DataFrame
test_data = pd.read_csv('data-train/test_data.csv')

In [11]:
train_data.sample(5)
train_data[train_data.transaction_id == "Transaction_1318448"]

,date,transaction_id,customer_id,product_id,has_loyality_card,store_id,is_promo,quantity,format,order_channel
7,2023-02-16,Transaction_1318448,Household_416,Product_21347,0,Store_2,0,2.0,DRIVE,WEBSITE
261946,2023-02-16,Transaction_1318448,Household_416,Product_12209,0,Store_2,0,1.0,DRIVE,WEBSITE
2643226,2023-02-16,Transaction_1318448,Household_416,Product_72783,0,Store_2,0,1.0,DRIVE,WEBSITE
3710620,2023-02-16,Transaction_1318448,Household_416,Product_71540,0,Store_2,0,2.0,DRIVE,WEBSITE
6102630,2023-02-16,Transaction_1318448,Household_416,Product_16529,0,Store_2,0,2.0,DRIVE,WEBSITE
7165314,2023-02-16,Transaction_1318448,Household_416,Product_5782,0,Store_2,0,1.0,DRIVE,WEBSITE
7960562,2023-02-16,Transaction_1318448,Household_416,Product_43653,0,Store_2,1,3.0,DRIVE,WEBSITE


In [7]:
products_data.sample(3)
products_data[products_data.product_id == "Product_21347"]

,product_id,product_description,department_key,class_key,subclass_key,sector,brand_key,shelf_level1,shelf_level2,shelf_level3,shelf_level4,bio,sugar_free,aspartame_free,gluten_free,halal,casher,eco_friendly,local_french,artificial_coloring_free,taste_enhancer_free,naturality,antibiotic_free,reduced_sugar,vegetarian,pesticide_free,grain_free,no_added_sugar,salt_reduced,nitrite_free,fed_without_ogm,no_added_salt,no_artificial_flavours,porc,vegan,frozen,fat_free,reduced_fats,fresh,alcool,lactose_free,phenylalanine_free,palm_oil_free,ecoscore,produits_du_monde,regional_product,national_brand,first_price_brand,carrefour_brand
8150,Product_21347,PAMPERS PP NB PAQUET T0 X22,Department_12,Class_1202,SubClass_12021,PGC,PAMPERS,Bébé,Change et Soins,Couches,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0


In [12]:
test_data.sample(3)

,transaction_id,customer_id,product_id
510231,Transaction_2024_34234,Household_20162,Product_1597
771865,Transaction_2024_29000,Household_71671,Product_17773
118812,Transaction_2024_29389,Household_44137,Product_22627


Creer un dataframe des fréquences

In [14]:
# Create a new DataFrame with customer_id, product_id, and count
frequency_df = train_data.groupby(['customer_id', 'product_id'])['transaction_id'].count().reset_index(name='frequency')
frequency_df.sample(4)

,customer_id,product_id,frequency
5824295,Household_763,Product_79728,1
5624004,Household_707,Product_68400,2
1343449,Household_13609,Product_16477,2
4236784,Household_3317,Product_44549,2


In [19]:
# Regrouper les données par client et produit, puis calculer le nombre de transactions et la quantité totale
frequency_quantity_df = train_data.groupby(['customer_id', 'product_id']).agg(
    frequency=('transaction_id', 'count'),  # Compter le nombre de transactions pour chaque produit par client
    total_quantity=('quantity', 'sum')      # Calculer la somme des quantités pour chaque produit par client
).reset_index()



In [15]:
# Rank the frequency for each customer in descending order
frequency_df['rank'] = frequency_df.groupby('customer_id')['frequency'].rank(ascending=False, method="first").astype("Int64")
frequency_df.head(4)

,customer_id,product_id,frequency,rank
0,Household_1,Product_10282,2,42
1,Household_1,Product_10303,1,110
2,Household_1,Product_10435,1,111
3,Household_1,Product_10469,1,112


In [20]:
frequency_quantity_df['score'] = (
    frequency_quantity_df['frequency'] + frequency_quantity_df['total_quantity']
)

# Ajouter un rang pour chaque produit par client en fonction du score d'importance
frequency_quantity_df['rank'] = frequency_quantity_df.groupby('customer_id')['score'].rank(ascending=False, method="first").astype("Int64")

# Afficher le résultat
print(frequency_quantity_df.head())


   customer_id     product_id  frequency  total_quantity  score  rank
0  Household_1  Product_10282          2             2.0    4.0    58
1  Household_1  Product_10303          1             1.0    2.0   173
2  Household_1  Product_10435          1             1.0    2.0   174
3  Household_1  Product_10469          1             1.0    2.0   175
4  Household_1  Product_10713          1             1.0    2.0   176


Regarder les résultats

In [34]:
frequency_df[frequency_df.customer_id == "Household_1"].sort_values(by = "rank").head(10)

test_freq = frequency_df[frequency_df.customer_id == "Household_1"]
test_freq[test_freq.product_id == "Product_45719"]

,customer_id,product_id,frequency,rank
272,Household_1,Product_45719,1,326


In [40]:
test_data[test_data.customer_id == "Household_1"]
#on retrouve les articles acheter par chaque utilisateurs qu'on est censé predire 


,customer_id,product_id,frequency
658469,Household_489,Product_15956,1
658470,Household_489,Product_19342,1
658471,Household_489,Product_29814,1
658472,Household_489,Product_31357,1
658473,Household_489,Product_31364,1
658474,Household_489,Product_36301,1
658475,Household_489,Product_4114,1
658476,Household_489,Product_41166,1
658477,Household_489,Product_47526,1
658478,Household_489,Product_58553,1


Créer un fichier avec les 10 recommandation par clients

In [25]:
# Here we keep only the top 10 recommendation and we drop the frequency as it will not be used in later steps
top_10_recommendations = frequency_df[frequency_df['rank'] <= 10].drop(columns = "frequency") 

In [ ]:
top_10_recommendations = frequency_quantity_df[frequency_quantity_df['rank'] <= 10].drop(columns = "frequency") 

In [26]:
# show the top 10 recommendations for customer Household_1 and Household_2

top_10_recommendations[top_10_recommendations.customer_id.isin(["Household_1", "Household_2"])].sort_values(by = ["customer_id", "rank"])

,customer_id,product_id,rank
347,Household_1,Product_57942,1
390,Household_1,Product_64067,2
421,Household_1,Product_67459,3
504,Household_1,Product_7783,4
97,Household_1,Product_21613,5
116,Household_1,Product_24334,6
541,Household_1,Product_833,7
440,Household_1,Product_7006,8
6,Household_1,Product_1128,9
54,Household_1,Product_170,10


Evaluer les résultats 

In [27]:
# Hitrate@10 evaluation function

def hitrate_at_k(true_data: pd.DataFrame,
                 predicted_data: pd.DataFrame,
                 k: int = 10) -> float:
    """
    This function calculates the hitrate at k for the recommendations.
    It assesses how relevant our 10 product recommendations are.
    In other words, it calculates the proportion of recommended products that are actually purchased by the customer.

    Args:
        true_data: a pandas DataFrame containing the true data
            customer_id: the customer identifier
            product_id: the product identifier that was purchased in the test set
        predicted_data: a pandas DataFrame containing the predicted data
            customer_id: the customer identifier
            product_id: the product identifier that was recommended
            rank: the rank of the recommendation. the rank should be between 1 and 10.
        k: the number of recommendations to consider. k should be between 1 and 10.
    
    Returns:
        The hitrate at k
    """
    
    data = pd.merge(left = true_data, right = predicted_data, how = "left", on = ["customer_id", "product_id"])
    df = data[data["rank"] <= k]
    non_null_counts = df.groupby('customer_id')['rank'].apply(lambda x: x.notna().sum()).reset_index(name='non_null_count')
    distinct_products_per_customer = data.groupby('customer_id')['product_id'].nunique().reset_index(name='distinct_product_count')
    df = pd.merge(left = distinct_products_per_customer, right = non_null_counts, how = "left", on = "customer_id")
    df["denominator"] = [min(df.iloc[i].distinct_product_count,k) for i in range(len(df))]
    df = df.fillna(0)
    return (df["non_null_count"]/df["denominator"]).mean()

In [28]:
# Calculate the hitrate at k for k = 10
frequency_model_hitrate_at_10 = hitrate_at_k(test_data,top_10_recommendations,10)
print(f"Hitrate@10 for the frequency model is {frequency_model_hitrate_at_10:.2f}")

Hitrate@10 for the frequency model is 0.08


# Faire un fichier pour la proposition sur kaggle

In [29]:
# Create submission file for 

# Keep only the top 10 recommendations for Households between 80001 and 100000
prediction = top_10_recommendations[
    top_10_recommendations.customer_id.isin(
            [
                f"Household_{i}" for i in range(80001,100001)
            ]
        )
    ]

# Print the solution
prediction.head()

,customer_id,product_id,rank


In [44]:
def process_and_format_prediction(df):
    # Remplacement des caractères invalides dans les noms de colonnes
    df.columns = df.columns.str.replace('+AF8-', '_', regex=False)
    df = df.replace(r'\+AF8-', '_', regex=True)

    # Nettoyage des colonnes 'customer_id', 'product_id', et 'transaction_id'
    if 'customer_id' in df.columns and df['customer_id'].dtype == 'object':
        df['customer_id'] = df['customer_id'].str.extract('(\d+)').fillna(11).astype(int)
    if 'product_id' in df.columns and df['product_id'].dtype == 'object':
        df['product_id'] = df['product_id'].str.extract('(\d+)').fillna(11).astype(int)
    if 'transaction_id' in df.columns and df['transaction_id'].dtype == 'object':
        df['transaction_id'] = df['transaction_id'].str.replace(r'\D', '', regex=True).fillna(11).astype(int)

    df['id'] = df.index
    df = df[['id'] + [col for col in df.columns if col != 'id']]

    if 'customer_id' not in df.columns or 'product_id' not in df.columns:
        raise ValueError("true_data must contain 'customer_id' and 'product_id' columns")

    # Grouper par customer_id et concaténer les valeurs des produits et des ranks
    prediction_grouped = df.groupby('customer_id').agg({
        'id': 'first',  # Prend la première valeur de 'id'
        'product_id': lambda x: ','.join(map(str, x)),  # Concatène les product_id en chaîne de caractères
        'rank': lambda x: ','.join(map(str, x))  # Concatène les ranks en chaîne de caractères
    }).reset_index()

    # Supprimer la colonne 'id' si elle existe
    if 'id' in prediction_grouped.columns:
        prediction_grouped = prediction_grouped.drop(columns=['id'])

    # Filtrer les données
    prediction_grouped = prediction_grouped[prediction_grouped['customer_id'] != 11]
    prediction_grouped.insert(0, 'id', range(len(prediction_grouped)))
    
       # Vérification des rangs et des doublons
    for index, row in prediction_grouped.iterrows():
        # Vérifier les ranks
        ranks = list(map(int, row['rank'].split(',')))
        if sorted(ranks) != list(range(1, 11)):  # Vérifie que les rangs sont distincts de 1 à 10
            print("Doublon détecté. Les rangs doivent être distincts (de 1 à 10) pour chacun des 10 produits prédits pour un client.\n")
            return None
        # Vérifier les doublons de produits
        products = row['product_id'].split(',')
        if len(products) != len(set(products)):  # Si des doublons sont présents dans les produits
            print("Doublon détecté. Il doit y avoir 10 produits différents par client.\n")
            return None
        

    return prediction_grouped
prediction_grouped=process_and_format_prediction(prediction)
print(prediction_grouped)

Empty DataFrame
Columns: [id, customer_id, product_id, rank]
Index: []
